<a href="https://colab.research.google.com/github/CraigDaGama/SmartED-AI/blob/main/FacialRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install opencv-python dlib face_recognition

In [34]:
import cv2
import face_recognition
import os
import numpy as np


In [35]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for subdir in os.listdir(folder):
        subdir_path = os.path.join(folder, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    images.append(img)
                    labels.append(subdir)
    return images, labels

In [36]:
known_faces_dir = 'known_faces'
known_images, known_labels = load_images_from_folder(known_faces_dir)

In [37]:
# Encode faces
known_face_encodings = []
known_face_names = []
for img, label in zip(known_images, known_labels):
 img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 encodings = face_recognition.face_encodings(img_rgb)
 if encodings:
  known_face_encodings.append(encodings[0])
  known_face_names.append(label)


In [38]:
def recognize_faces_in_video(known_face_encodings, known_face_names):
    video_capture = cv2.VideoCapture(0)
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        # Convert the frame to RGB
        rgb_frame = frame[:, :, ::-1]
        # Find all face locations and face encodings in the frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        # Loop through each face in this frame
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)
        # Display the resulting image
        cv2.imshow('Video', frame)
        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [39]:
recognize_faces_in_video(known_face_encodings, known_face_names)